In [10]:
# Define a function to get latitude-longitude pairs for a given address
def get_latlon(address):
    geocode_result = gmaps.geocode(address)
    if geocode_result:
        lat = geocode_result[0]['geometry']['location']['lat']
        lon = geocode_result[0]['geometry']['location']['lng']
        return lat, lon
    else:
        return None

# Replace "Name" values in t with latitude-longitude pairs from connections
for idx, row in connections.iterrows():
    node = row['Node']
    edge = row['Edge']
    node_latlon = get_latlon(node)
    edge_latlon = get_latlon(edge)
    if node_latlon and edge_latlon:
        t.loc[t['Name'] == node, 'Name'] = f"{node_latlon[0]},{node_latlon[1]}"
        t.loc[t['Name'] == edge, 'Name'] = f"{edge_latlon[0]},{edge_latlon[1]}"

# Save the modified GeoDataFrame to a new shapefile
output_file = 'ModifiedTracts.shp'
t.to_file(output_file)


KeyError: 'Node'

In [ ]:


# Function to calculate travel time between node-edge pairs
def calculate_travel_time(node_edge_pairs, api_key):
    travel_times = {}
    for pair in node_edge_pairs:
        start_latlon = f"{pair[0][0]},{pair[0][1]}"
        end_latlon = f"{pair[1][0]},{pair[1][1]}"
        directions = gmaps.directions(start_latlon, end_latlon, mode="driving")
        if directions:
            travel_time_sec = directions[0]['legs'][0]['duration']['value']
            travel_times[pair] = travel_time_sec
    return travel_times

# Calculate travel times
travel_times = calculate_travel_time(node_edge_pairs, api_key)

# Print travel times
for pair, time in travel_times.items():
    print(f"Node-Edge Pair: {pair}, Travel Time (seconds): {time}")
